In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))
source(paste0(HELP_DIR, "fisher.r"))

In [ ]:
library(patchwork)

# Prepare data for analysis

- Maps

In [ ]:
cohorts_map <- fread(paste0(SHARE_DIR, "cohort_names_map.csv"))
treatment_mechanism_map <- fread(paste0(SHARE_DIR, "treatment_mechanism_map.csv"))

- Derive results

In [ ]:
nr_threshold <- .05
fdr_threshold <- .1

In [ ]:
go_results <- 
fread(paste0(SHARE_DIR, "2b_go.csv")) %>% 
 #fi( (grepl("_eq", feature) | grepl("_gt", feature) | grepl("_lt", feature)) & (direction == "Non-Response")) %>% 
 fi(group != "type", !grepl("_or_", feature)) %>%  ### 
 mu(or = ifelse(or == "Inf", exp(5), or), 
    p_bh = p.adjust(fisher_pval, method = "BH"),  
    p_by = p.adjust(fisher_pval, method = "BY"), 
    q = p.adjust(fisher_pval, method = "fdr")) %>% 
 rw() %>% mu( derived_treatmentName = str_split_fixed( cohortGo	, " ## ", n = 2)[2]) %>% ug() %>% 
 lj( treatment_mechanism_map , by = "derived_treatmentName") %>% 
 mu( derived_treatmentMechanism = 
     ifelse(is.na(derived_treatmentMechanism), derived_treatmentName, derived_treatmentMechanism),
     treatment = derived_treatmentName, mechanism = gsub(" ## ", "/", derived_treatmentMechanism),
     alpha_gp = case_when(p_bh >= fdr_threshold ~ "none", p_bh < nr_threshold ~ "dark"), 
     highlight = ((e_nr/events >= 1 - nr_threshold) & (p_bh < fdr_threshold)),
     color_gp = case_when(highlight ~ mechanism,!highlight & (p_bh < .1) ~ "significant",TRUE ~ "none"))

In [ ]:
fwrite(go_results %>% fi(highlight), "share_with_fran.csv")

# Volcano Plots

In [ ]:
options(repr.plot.height = 1, repr.plot.width = 6) 
my_colors <- c("#F04437", "#E81F64", "#903E97", "#65499E", "#4356A5", "#478FCC", "#34A4DD", "#00BCD4", "#009889", "#4BB04F", "#8BC34C", "#CCDA3A", "#FCED3A", "#FFC10E", "#F8991D", "#F1592C", "#7A5649", "#9F9E9E", "#607F8C")
df <- data.frame(color = my_colors,x = seq_along(my_colors))
ggplot(df, aes(x = factor(x), y = 1, fill = color)) + geom_tile() + scale_fill_identity() +theme_minimal() 

- Theme

In [ ]:
extra_theme <- 
theme(axis.text.x = element_text(angle = 0, size = 12), axis.text.y = element_text(size = 12), plot.title = element_text(size = 16)) 

- Multiple testing threshold

In [ ]:
threshold <- go_results %>% gb(mt = p_bh < fdr_threshold) %>% su(threshold = max(fisher_pval)) %>% fi(mt) %>% ug() %>% pu(threshold)

- Color, size, alpha map

In [ ]:
alpha_map <- list("none" = .01, "dark" = 1)
size_map <- list("TRUE" = 4, "FALSE" = 1)

In [ ]:
color_map <- 
list(
'none' = my_colors[18],
'significant' = my_colors[18],
'Anti-PD-1' = my_colors[1], 
'Multikinase inhibitor' = my_colors[2],
'Anti-AR' = my_colors[3],     
'Anti-EGFR' = my_colors[4], 
'Aromatase inhibitor' = my_colors[5],
'CAPEOX + Bevacizumab'  = my_colors[6],
'Anti-VEGF/Platinum/Pyrimidine (ant)agonist'  = my_colors[7],   
'Alkaloid/Platinum' = my_colors[8],
'Platinum/Pyrimidine (ant)agonist'= my_colors[9],
'Platinum/Taxane'= my_colors[10],
'Pyrimidine (ant)agonist'= my_colors[11],
'Taxane'= my_colors[12] ) 

- Make the plot

In [ ]:
volcano <- function(df, title = "here"){
 df %>% 
   ggplot( aes(x = e_r/events, y = -log10(fisher_pval), alpha = alpha_gp, color = color_gp, size = highlight)) + 
   geom_point() + 
   scale_alpha_manual(values = unlist(alpha_map)) + 
   scale_size_manual(values = unlist(size_map)) + 
   scale_color_manual(values = unlist(color_map)) +  
   go_theme + extra_theme + 
   geom_hline(yintercept = -log10(threshold), alpha = .7, color = "red") +
   geom_vline(xintercept = .05, alpha = .7, size = .1) + 
   labs(x = "Estimated Probability of Response", y = "Fisher's Test\n-Log10 (p-value)", 
        title = title, size = NULL, alpha = NULL, color = NULL) + 
  guides(alpha = "none", size = "none") + 
  scale_x_continuous( labels = percent_format(accuracy = 1), limits = c(0,1)) 
}

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 6) 
a <- volcano(go_results %>% fi(type == "univariate"), "Tests of Univariate Features")
b <- volcano(go_results %>% fi(type != "univariate"), "Tests of Combination Features")
c <- volcano(go_results, "Systematic Tests of Features")

# Send the figures

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 6) 
a
ggsave(paste0(FIG_DIR, "volcano_univariate.png"), width = 6, height = 4)

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 7) 
b
ggsave(paste0(FIG_DIR, "volcano_combination.png"), width = 7, height = 4)

In [ ]:
FIG_DIR

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 7) 
c
ggsave(paste0(FIG_DIR, "volcano_all.png"), width = 7, height = 4)